### Practice: Large Language Models and Their Implications

In this notebook, you're gonna play with some of the largest language models on the Internet.


### Local inference

In [3]:
!pip install --quiet bitsandbytes transformers sentencepiece optimum auto-gptq
!pip install -q -U mlx-lm accelerate>=0.26.0
!pip install bitsandbytes
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import bitsandbytes as bnb
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
model_name = 'TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T'

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    dtype=torch.float16,
    low_cpu_mem_usage=True#,
    #offload_state_dict=True
)

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

## Text generation

**Comparison of strategies for language model text generation:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

### Generation with HuggingFace

In [7]:
prompt = 'The first discovered martian lifeform looks like'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, max_new_tokens=64, do_sample=True, temperature=0.8)
# greedy inference:                                        do_sample=False)
# beam search for highest probability:                     num_beams=4)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))

Input batch (encoded): {'input_ids': tensor([[    1,   450,   937, 10943, 14436,   713,  2834,   689,  3430,   763]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Output: <s> The first discovered martian lifeform looks like a hoodie
Mars 2020 rover Curiosity: What you need to know</s>


In [8]:
prompt = "Moscow is the capital of"
# prompt = "Skippy, a young android, likes to dream about electric"

print(prompt, '\n')

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}  # reverse vocab for decode

for i in range(10):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    prompt += next_token

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f"Step #{i} candidates:")
    for t, p in zip (top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t:<10}: {p:.4f} ")

    print(f'\nChosen token: {next_token}', end='\n\n', flush=True)

Moscow is the capital of 

Step #0 candidates:
▁Russia   : 0.6587 
▁the      : 0.2224 
▁Russian  : 0.0150 
▁Moscow   : 0.0121 
▁a        : 0.0066 

Chosen token: the

Step #1 candidates:
▁Russian  : 0.4143 
▁country  : 0.0992 
▁federal  : 0.0676 
▁Republic : 0.0310 
▁republic : 0.0234 

Chosen token: Federal

Step #2 candidates:
▁Republic : 0.3904 
▁District : 0.0489 
▁S        : 0.0236 
▁State    : 0.0196 
S         : 0.0177 

Chosen token: District

Step #3 candidates:
▁of       : 0.3843 
,         : 0.1360 
▁and      : 0.0793 
of        : 0.0745 
.         : 0.0700 

Chosen token: of

Step #4 candidates:
M         : 0.2484 
Russ      : 0.2407 
the       : 0.0638 
▁the      : 0.0512 
K         : 0.0470 

Chosen token: Russ

Step #5 candidates:
ia        : 0.9072 
ian       : 0.0818 
ial       : 0.0013 
i         : 0.0012 
iah       : 0.0006 

Chosen token: ia

Step #6 candidates:
,         : 0.3325 
.         : 0.2375 
▁and      : 0.1030 
and       : 0.0490 
▁with     : 0.0332 

Chos

#### Nucleus sampling generation

In [9]:
from typing import Tuple, List
def nucleus_sampling(model, tokenizer, prompt: str, prob: float = 0.5) -> Tuple[str, List[str]]:
    """generates the next token from the nucleus of tokens with cumulative probability up to param:prob"""
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    logits = model.forward(**inputs).logits[0, -1, :]

    probs = torch.nn.functional.softmax(logits, dim=-1)

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    probs_cumsum = torch.cumsum(sorted_probs, dim=0)
    select_index = torch.where(probs_cumsum > prob)[0][0]

    if select_index in [0, 1]:
        sampled_token = tokenizer.decode(sorted_indices[0])
        possible_tokens = sampled_token
    else:
        possible_tokens_ids = sorted_indices[:select_index]
        possible_tokens_probs = sorted_probs[:select_index]

        sampled_token_id = torch.multinomial(possible_tokens_probs.flatten(), num_samples=1)
        sampled_token = tokenizer.decode(possible_tokens_ids[sampled_token_id])

        possible_tokens = tokenizer.decode(possible_tokens_ids).split(' ')

    # sampled_token should be a string token that was generated
    # possible_tokens should be a list of all tokens that have non-zero probability
    return sampled_token, possible_tokens

In [10]:
test_prompt = "Elbrus is the highest"
next_token, possible_tokens = nucleus_sampling(model, tokenizer, test_prompt, prob=0.9)
print(test_prompt, next_token, possible_tokens)

test_prompt = "Large language models can learn to"
next_token, possible_tokens = nucleus_sampling(model, tokenizer, test_prompt, prob=0.4)
print(test_prompt, next_token, possible_tokens)

Elbrus is the highest peak ['mountain', 'peak']
Large language models can learn to predict ['understand', 'predict', 'generate', 'recognize', 'translate', 'm', 'perform', 'do', 'model', 'produce', 'general', 'read']
